In [ ]:
import netCDF4 as nc
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

def find_lat_lon(dataset):
    """
    Find latitude and longitude variables in the dataset.

    Parameters:
    dataset (Dataset): The NetCDF dataset object.

    Returns:
    tuple: Latitude and longitude arrays.
    """
    lat = lon = None

    possible_groups = ['', 'data_01', 'data_01/ku']
    possible_lat_names = ['lat', 'latitude']
    possible_lon_names = ['lon', 'longitude']

    for group in possible_groups:
        try:
            group_obj = dataset[group] if group else dataset
            for lat_name in possible_lat_names:
                if lat_name in group_obj.variables:
                    lat = group_obj.variables[lat_name][:]
                    break
            for lon_name in possible_lon_names:
                if lon_name in group_obj.variables:
                    lon = group_obj.variables[lon_name][:]
                    break
        except KeyError:
            continue
        if lat is not None and lon is not None:
            break

    if lat is None or lon is None:
        raise ValueError("Latitude and/or Longitude variables not found in the dataset.")

    return lat, lon

def plot_lat_lon_points(nc_file, custom_point, bbox):
    """
    Plot latitude and longitude points from a NetCDF file and a custom point.

    Parameters:
    nc_file (str): Path to the NetCDF file.
    custom_point (tuple): Coordinates of the custom point as (lat, lon).
    bbox (tuple): Bounding box for the plot as (min_lon, max_lon, min_lat, max_lat).
    """
    # Open the NetCDF file
    dataset = nc.Dataset(nc_file)

    # Find latitude and longitude variables
    lat, lon = find_lat_lon(dataset)

    # Create a figure and axis with cartopy
    fig = plt.figure(figsize=(10, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())

    # Set the extent (bounding box) of the plot
    ax.set_extent(bbox, crs=ccrs.PlateCarree())

    # Add geographic features for better visualization
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN, facecolor='lightblue')

    # Plot the latitude and longitude points
    ax.scatter(lon, lat, color='blue', s=10, transform=ccrs.PlateCarree(), label='Satellite Track')

    # Plot the custom point
    custom_lat, custom_lon = custom_point
    ax.plot(custom_lon, custom_lat, 'o', color='white', markersize=12, markeredgecolor='black', transform=ccrs.PlateCarree(), label='NOAA Station')

    # Add a legend
    ax.legend(loc='upper right')

    # Add a grid
    gl = ax.gridlines(draw_labels=True)
    gl.right_labels = False
    gl.top_labels = False

    # Show the plot
    plt.title('Sentinel-6 track on August 30, 2023 and NOAA Cedar Key station')
    plt.show()

# Example usage
nc_file = '/work2/07174/soelem/data/datasentinel/S6A_P4_2__HR_STD__NT_103_102_20230830T001837_20230830T011450_F08.nc'
custom_point = (29.135, -83.0317)  # Example coordinates (latitude, longitude)
bbox = (-100, -60, 8, 46)  # Example bounding box (min_lon, max_lon, min_lat, max_lat)

plot_lat_lon_points(nc_file, custom_point, bbox)